### P0 Code Generator for WASM

#### Emil Sekerinski, revised March 2020


### Revised by group 14,  March 2021
##### Modificaitons:


* 1. Modify `genProgStart` by initializing write and read method for int, float and double
* 2. Add new generation procedures: `genDouble`, `genFloat` and modify `genRead` and `genWrite` due to different types
* 3. Modify `genGlobalVars`, `genLocalVars`, `genProcStart` due to the two new added type `Double` and `Float`
* 4. Modify procedure `genAssign(x, y)` depending on the type of x
* 5. Modify procedure `loadItem(x, typ = None)`, to support truncation (float or double to int) and demotion (double to float)
* 6. Modify procedure `genUnaryOp` to supports explicit type conversion
* 7. Modify procedure `genBinaryOp` and `genRelation`to supports operands with differetn types
* 8. Modify `genCall` to check the return type of called procedures
* 9. add `genAccumulation(op, x, y, z)` to calculate the sum or maximum value of an array with a range of index
* 10. Modify unaryOP() to add SQRT, LEFTCEIL, LEFTFLOOR, ABSVALUE

In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from SC import TIMES, DIV, MOD, AND, PLUS, MINUS, OR, EQ, NE, LT, GT, LE, GE, \
     NOT, CARD, COMPLEMENT, UNION, INTERSECTION, ELEMENT, SUBSET, SUPERSET, SET, \
     mark, CONVERT_TO_INT, CONVERT_TO_FLOAT, CONVERT_TO_DOUBLE, SUM, MAX, \
     SQRT, LEFTCEIL, LEFTFLOOR, ABSVALUE, MIN
from ST import indent, Var, Const, Type, Proc, StdProc, Int, Bool, Array, \
     Record, Set, Double, Float

Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb


In [2]:
def genProgStart():
    global curlev, memsize, asm
    curlev, memsize = 0, 0
    asm = ['(module',
           '(import "P0lib" "writeInt" (func $writeInt (param i32)))',
           '(import "P0lib" "writeDouble" (func $writeDouble (param f64)))',
           '(import "P0lib" "writeFloat" (func $writeFloat (param f32)))',
           '(import "P0lib" "writeln" (func $writeln))',
           '(import "P0lib" "readInt" (func $readInt (result i32)))',
           '(import "P0lib" "readDouble" (func $readDouble (result f64)))',
           '(import "P0lib" "readFloat" (func $readFloat (result f32)))']

Add new generation procedures for float type and double type
- Doubles occupy 8 bytes
- Floats occupy 4 bytes

In [3]:
def genBool(b):
    # b is Bool
    b.size = 1; return b

def genInt(i):
    # i is Int
    i.size = 4; return i

# generation procedure for double type
def genDouble(i):
    # i is Double
    i.size = 8; return i
# generation procedure for float type
def genFloat(i):
    # i is Float
    i.size = 4; return i

def genRec(r: Record):
    s = 0
    for f in r.fields:
        f.offset, s = s, s + f.tp.size
    r.size = s
    return r

def genArray(a: Array):
    a.size = a.length * a.base.size
    return a

def genSet(s: Set):
    if s.lower < 0 or s.lower + s.length > 32:
        mark('WASM: set too large')
    s.size = 4; return s

In [4]:
Global = 0; Stack = -1; MemInd = -2; MemAbs = -3

If the type is `Float`, allocates a global WebAssembly variable by f32, if the type is `Double`, allocates a global WebAssembly variable by f64

In [5]:
def genGlobalVars(sc, start):
    global memsize
    for i in range(start, len(sc)):
        if type(sc[i]) == Var:
            if sc[i].tp in (Int, Bool) or type(sc[i].tp) == Set:
                asm.append('(global $' + sc[i].name + ' (mut i32) i32.const 0)')
            # if the variable is float type, allocate a gloable WebAssembly variable by f32 and initalize its value to be 0
            elif sc[i].tp == Float: 
                asm.append('(global $' + sc[i].name + ' (mut f32) f32.const 0.0)')
            # if the variable is double type, allocate a gloable WebAssembly variable by f64 and initalize its value to be 0
            elif sc[i].tp == Double: 
                asm.append('(global $' + sc[i].name + ' (mut f64) f64.const 0.0)')    
            elif type(sc[i].tp) in (Array, Record):
                sc[i].lev, sc[i].adr, memsize = MemAbs, memsize, memsize + sc[i].tp.size
            else: mark('WASM: type?')
    
def genLocalVars(sc, start):
    for i in range(start, len(sc)):
        if type(sc[i]) == Var:
            # if the variable is float type, allocate a local WebAssembly variable by f32
            if sc[i].tp == Float: 
                asm.append('(local $' + sc[i].name + ' f32)')
            # if the variable is double type, allocate a local WebAssembly variable by f64
            elif sc[i].tp == Double: 
                asm.append('(local $' + sc[i].name + ' f64)')
            # if the variable is int or array, allocate a local WebAssembly variable by i32
            else: 
                asm.append('(local $' + sc[i].name + ' i32)')
    asm.append('(local $0 i32)') # auxiliary local variable
    return sc[start:]

Procedure `loadItem(x)` generates code for loading `x` on the expression stack, assuming `x` is global `Var`, local `Var`, stack `Var`, memory `Var`, local `Ref`, stack `Ref`, `Const`.
For the memroty `Var`, local `Ref`, stack `Ref`, `Const`, it should generate code depending on the variable types

In [1]:
# in the case of assignment or binary operands of different types,
# loadItem is passed the parameter typ which indicates the new type
# conversions may go from higher precision to lower precision with trunc or demote
# or from lower precision to higher precision with convert, or promote
def loadItem(x, typ=None):
    if type(x) == Var:
        if x.lev == Global: 
            asm.append('global.get $' + x.name) # global Var
            if x.tp == Int and typ == Float: asm.append('f32.convert_i32_s')
            elif x.tp == Int and typ == Double: asm.append('f64.convert_i32_s')
            elif x.tp == Float and typ == Double: asm.append('f64.promote_f32')
            elif x.tp == Float and typ == Int: asm.append('i32.trunc_f32_s')
            elif x.tp == Double and typ == Int: asm.append('i32.trunc_f64_s')
            elif x.tp == Double and typ == Float: asm.append('f32.demote_f64')
        elif x.lev == curlev: 
            asm.append('local.get $' + x.name) # local Var
            if x.tp == Int and typ == Float: asm.append('f32.convert_i32_s')
            elif x.tp == Int and typ == Double: asm.append('f64.convert_i32_s')
            elif x.tp == Float and typ == Double: asm.append('f64.promote_f32')
            elif x.tp == Float and typ == Int: asm.append('i32.trunc_f32_s')
            elif x.tp == Double and typ == Int: asm.append('i32.trunc_f64_s')
            elif x.tp == Double and typ == Float: asm.append('f32.demote_f64')
        elif x.lev == MemInd: 
            if x.tp in (Int, Bool):
                asm.append('i32.load')
            elif x.tp == Double:
                asm.append('f64.load')
            elif x.tp == Float:
                asm.append('f32.load')
        elif x.lev == MemAbs:
            asm.append('i32.const ' + str(x.adr))
            if x.tp in {Int, Bool, Array}: 
                asm.append('i32.load')
            elif x.tp == Double:
                asm.append('f64.load')
            elif x.tp == Float:
                asm.append('f32.load')
        elif x.lev != Stack: mark('WASM: var level!') # already on stack if lev == -1
    else: 
        if x.tp in (Int, Bool):
            asm.append('i32.const ' + str(x.val))
            if typ == Float: asm.append('f32.convert_i32_s') 
            elif typ == Double: asm.append('f64.convert_i32_s')
        elif x.tp == Double:
            asm.append('f64.const ' + str(x.val))
            if typ == Int: asm.append('i32.trunc_f64_s')
            elif typ == Float: asm.append('f32.demote_f64')
        elif x.tp == Float:
            asm.append('f32.const ' + str(x.val))
            if typ == Int: asm.append('i32.trunc_f32_s')
            elif typ == Double: asm.append('f64.promote_f32')

In [7]:
def genVar(x):
    if Global < x.lev < curlev: mark('WASM: level!')
    y = Var(x.tp); y.lev, y.name = x.lev, x.name
    if x.lev == MemAbs: y.adr = x.adr
    return y

In [8]:
def genConst(x):
    # x is Const
    x.lev = None # constants are either not stored or on stack, lev == Stack
    return x

Procedure `genUnaryOp(op, x)` generates code for `op x` if `op` is `MINUS`, `NOT`, `AND`, `OR` and `x` is `Int`,  `Double`,  `Float`, `Bool`, respectively. After Adding `Double` and `Float` type, we need to check the type of x such that generate different code based on the type of x.

In [9]:
def genUnaryOp(op, x):
    loadItem(x)
    if op == MINUS:# check the type of x
        if x.tp == Int: # if x is Int
            asm.append('i32.const -1')
            asm.append('i32.mul')
            x = Var(Int); x.lev = Stack
        elif x.tp == Double:# if x is Double
            asm.append('f64.const -1.0')
            asm.append('f64.mul')
            x = Var(Double); x.lev = Stack
        elif x.tp == Float:# if x is Float
            asm.append('f32.const -1.0')            
            asm.append('f32.mul')
            x = Var(Float); x.lev = Stack
    elif op == CARD:
        asm.append('i32.popcnt')
        x = Var(Int); x.lev = Stack
    elif op == COMPLEMENT:
        u = (1 << x.tp.length) - 1 # x.tp.length 1's
        u = u << x.tp.lower # universe of base type
        asm.append('i32.const ' + hex(u))
        asm.append('i32.xor')
        x = Var(x.tp); x.lev = Stack
    elif op == SET:
        asm.append('local.set $0')
        asm.append('i32.const 1')
        asm.append('local.get $0')
        asm.append('i32.shl')
        x = Var(Set(0, 32)); x.lev = Stack
    elif op == NOT:
        asm.append('i32.eqz')
        x = Var(Bool); x.lev = Stack
    elif op == AND:
        asm.append('if (result i32)')
        x = Var(Bool); x.lev = Stack
    elif op == OR:
        asm.append('if (result i32)')
        asm.append('i32.const 1')
        asm.append('else')
        x = Var(Bool); x.lev = Stack
    elif op == ELEMENT:
        asm.append('local.set $0')
        asm.append('i32.const 1')
        asm.append('local.get $0')
        asm.append('i32.shl')
        x = Var(Int); x.lev = Stack
    elif op in {SUBSET, SUPERSET}:
        asm.append('local.tee $0')
        asm.append('local.get $0')
        x.lev = Stack
    elif op == CONVERT_TO_INT:# convert to Int
        if x.tp == Double: # if x is Double
            asm.append('i32.trunc_f64_s')
        elif x.tp == Float:# if x is Float
            asm.append('i32.trunc_f32_s')
        x = Var(Int); x.lev = Stack
    elif op == CONVERT_TO_FLOAT:# convert to Float
        if x.tp == Int: # if x is Int
            asm.append('f32.convert_i32_s')
        elif x.tp == Double:# if x is Double
            asm.append('f32.demote_f64')
        x = Var(Float); x.lev = Stack
    elif op == CONVERT_TO_DOUBLE:# convert to Double
        if x.tp == Int: # if x is Int
            asm.append('f64.convert_i32_s')
        elif x.tp == Float:# if x is Float
            asm.append('f64.promote_f32')
        x = Var(Double); x.lev = Stack
    # the SQRT, FLOOR, CEILING, and ABSVALUE operations 
    # call the sqrt, floor, ceil, abs WASM operators, 
    # for float or double precision but not integer
    elif op == SQRT:
        if x.tp == Float:
            asm.append('f32.sqrt')
            x = Var(Float); x.lev = Stack
        elif x.tp == Double:
            asm.append('f64.sqrt')
            x = Var(Double); x.lev = Stack
    elif op == LEFTFLOOR:
        if x.tp == Float:
            asm.append('f32.floor')
            x = Var(Float); x.lev = Stack
        elif x.tp == Double:
            asm.append('f64.floor')
            x = Var(Double); x.lev = Stack
    elif op == LEFTCEIL:
        if x.tp == Float:
            asm.append('f32.ceil')
            x = Var(Float); x.lev = Stack
        elif x.tp == Double:
            asm.append('f64.ceil')
            x = Var(Double); x.lev = Stack
    elif op == ABSVALUE:
        if x.tp == Float:
            asm.append('f32.abs')
            x = Var(Float); x.lev = Stack
        elif x.tp == Double:
            asm.append('f64.abs')
            x = Var(Double); x.lev = Stack
    else: mark('WASM: unary operator?')
    return x

If `op` is `PLUS`, `MINUS`, `TIMES`, `DIV`, `MAX`, `MIN` check the type of x, and generate different code based on the type of x.
If `op` is `Mod`, geneate code for Int type since this operation is not applicable for Double and Float.

In [10]:
def genBinaryOp(op, x, y):
    # the PLUS, MINUS, TIMES, and DIV operations are supported
    # by WASM for floats and doubles
    # in the case of operands of different types,
    # the lower precision operand is converted to the higher
    if op in (PLUS, MINUS, TIMES, DIV, MOD):
        if x.tp == Int == y.tp: 
            loadItem(x); loadItem(y)
            asm.append('i32.add' if op == PLUS else \
                   'i32.sub' if op == MINUS else \
                   'i32.mul' if op == TIMES else \
                   'i32.div_s' if op == DIV else \
                   'i32.rem_s' if op == MOD else '?')
            x = Var(Int); x.lev = Stack
        elif x.tp == Double == y.tp: # if x is Double
            loadItem(x); loadItem(y)
            asm.append('f64.add' if op == PLUS else \
                   'f64.sub' if op == MINUS else \
                   'f64.mul' if op == TIMES else \
                   'f64.div' if op == DIV else '?')
            x = Var(Double); x.lev = Stack
        elif x.tp == Float == y.tp: # if x is Float
            loadItem(x); loadItem(y)
            asm.append('f32.add' if op == PLUS else \
                   'f32.sub' if op == MINUS else \
                   'f32.mul' if op == TIMES else \
                   'f32.div' if op == DIV else '?')
            x = Var(Float); x.lev = Stack
        elif (x.tp == Int and y.tp == Float) or (x.tp == Float and y.tp == Int):
            if x.tp == Int: 
                loadItem(x, Float); loadItem(y)
            else: loadItem(x); loadItem(y, Float)
            asm.append('f32.add' if op == PLUS else \
                   'f32.sub' if op == MINUS else \
                   'f32.mul' if op == TIMES else \
                   'f32.div' if op == DIV else '?')
            x = Var(Float); x.lev = Stack
        elif ((x.tp == Int or x.tp == Float) and y.tp == Double) or (x.tp == Double and (y.tp == Int or y.tp == Float)):
            if x.tp == Double: loadItem(x); loadItem(y, Double)
            else: loadItem(x, Double); loadItem(y)
            asm.append('f64.add' if op == PLUS else \
                   'f64.sub' if op == MINUS else \
                   'f64.mul' if op == TIMES else \
                   'f64.div' if op == DIV else '?')
            x = Var(Double); x.lev = Stack    
    elif op in {UNION, INTERSECTION}:
        loadItem(x); loadItem(y)
        asm.append('i32.or' if op == UNION else \
                   'i32.and' if op == INTERSECTION else '?')
        x = Var(x.tp); x.lev = Stack
    elif op == AND:
        loadItem(y) # x is already on the stack
        asm.append('else')
        asm.append('i32.const 0')
        asm.append('end')
        x = Var(Bool); x.lev = Stack
    elif op == OR:
        loadItem(y) # x is already on the stack
        asm.append('end')
        x = Var(Bool); x.lev = Stack
    elif op in (MAX, MIN):
        if x.tp == Double == y.tp: # if both are Double
            loadItem(x); loadItem(y)
            asm.append('f64.max' if op == MAX else \
                   'f64.min' if op == MIN else '?')
            x = Var(Double); x.lev = Stack
        elif x.tp == Float == y.tp: # if both are Float
            loadItem(x); loadItem(y)
            asm.append('f32.max' if op == MAX else \
                   'f32.min' if op == MIN else '?')
            x = Var(Float); x.lev = Stack
        elif x.tp == Int == y.tp: # if both are Int
            loadItem(x, Float); loadItem(y, Float)
            asm.append('f32.max' if op == MAX else \
                   'f32.min' if op == MIN else '?')
            x = Var(Float); x.lev = Stack
        elif (x.tp == Int and y.tp == Float) or (x.tp == Float and y.tp == Int): #one is float and one is int
            if x.tp == Int: 
                loadItem(x, Float); loadItem(y)
            else: loadItem(x); loadItem(y, Float)
            asm.append('f32.max' if op == MAX else \
                   'f32.min' if op == MIN else '?')
            x = Var(Float); x.lev = Stack
        elif ((x.tp == Int or x.tp == Float) and y.tp == Double) or (x.tp == Double and (y.tp == Int or y.tp == Float)):
            if x.tp == Double: loadItem(x); loadItem(y, Double)
            else: loadItem(x, Double); loadItem(y)
            asm.append('f64.max' if op == MAX else \
                   'f64.min' if op == MIN else '?')
            x = Var(Double); x.lev = Stack        
    else: mark('WASM: binary operator?')
    return x

Procedure `genRelation(op, x, y)` generates different code for `x op y` if `op` is `EQ`, `NE`, `LT`, `LE`, `GT`, `GE` depending on the type of x

In [11]:
def genRelation(op, x, y):
    if op in (EQ, NE, LT, LE, GT, GE):
        if x.tp == Double or y.tp == Double:# if x or y is Double
            loadItem(x, Double); loadItem(y, Double)
            asm.append('f64.eq' if op == EQ else \
                   'f64.ne' if op == NE else \
                   'f64.lt' if op ==  LT else \
                   'f64.gt' if op == GT else \
                   'f64.le' if op == LE else \
                   'f64.ge' if op == GE else '?')  
        elif (x.tp == Float and y.tp in (Int, Float)) or (y.tp == Float and x.tp in (Int, Float)):# if maximum type is Float
            loadItem(x, Float); loadItem(y, Float)
            asm.append('f32.eq' if op == EQ else \
                   'f32.ne' if op == NE else \
                   'f32.lt' if op ==  LT else \
                   'f32.gt' if op == GT else \
                   'f32.le' if op == LE else \
                   'f32.ge' if op == GE else '?')             
        else: # if x and y are both Int
            loadItem(x); loadItem(y)
            asm.append('i32.eq' if op == EQ else \
                   'i32.ne' if op == NE else \
                   'i32.lt_s' if op ==  LT else \
                   'i32.gt_s' if op == GT else \
                   'i32.le_s' if op == LE else \
                   'i32.ge_s' if op == GE else '?')        
    else:
        asm.extend('i32.and' if op == ELEMENT else \
               'i32.and', 'i32.eq' if op == SUBSET else \
               'i32.or', 'i32.eq' if op == SUPERSET else '?')
    x = Var(Bool); x.lev = Stack
    return x

In [12]:
def genAccumulation(op, x, y, z):
    #x[y:z], assuming x.tp is Array and x is global Var, local Var
    # y and z is Const, local Var, global Var, stack Var
    if x.lev == MemAbs and type(y) == Const and type(z) == Const: 
        x.adr += (y.val - x.tp.lower) * x.tp.base.size
        asm.append('i32.const ' + str(x.adr))
        if x.tp.base == Int:            
            asm.append('i32.load')
            if(op == MAX): asm.append('f32.convert_i32_s')
        elif x.tp.base == Float:
            asm.append('f32.load')
        elif x.tp.base == Double:
            asm.append('f64.load')
        i = 1; n = z.val - y.val
        while(i < n):
            asm.append('i32.const ' + str(x.adr + i * x.tp.base.size))
            if x.tp.base == Int: 
                asm.append('i32.load')
                if op == SUM: asm.append('i32.add')
                if op == MAX: 
                    asm.append('f32.convert_i32_s') 
                    asm.append('f32.max')
            elif x.tp.base == Float:
                asm.append('f32.load')
                if op == SUM: asm.append('f32.add')
                elif op == MAX: asm.append('f32.max')
            elif x.tp.base == Double:
                asm.append('f64.load') 
                if op == SUM: asm.append('f64.add')
                elif op == MAX: asm.append('f64.max')
            i = i + 1
        if x.tp.base == Int and op == MAX: asm.append('i32.trunc_f32_s')           
        x = Var(x.tp.base); x.lev = Stack
    else:
        # get the addres of the first element of x[y: z] and load the first emlement to the stack
        loadItem(y)
        if x.tp.lower != 0:
            asm.append('i32.const ' + str(x.tp.lower))
            asm.append('i32.sub')
        asm.append('i32.const ' + str(x.tp.base.size))
        asm.append('i32.mul')
        if x.lev > 0: asm.append('local.get $' + x.name)
        elif x.lev == MemAbs: asm.append('i32.const ' + str(x.adr))
        asm.append('i32.add')
        if x.tp.base == Int: 
            asm.append('i32.load')
            if op == MAX: asm.append('f32.convert_i32_s') 
        elif x.tp.base == Float:
            asm.append('f32.load')
        elif x.tp.base == Double:
            asm.append('f64.load')
        i = 1; n = z.val - y.val        
        while(i < n):
                # get the address of the next element and load it to the stack
                y.val = y.val + 1
                loadItem(y)
                if x.tp.lower != 0:
                    asm.append('i32.const ' + str(x.tp.lower))
                    asm.append('i32.sub')
                asm.append('i32.const ' + str(x.tp.base.size))
                asm.append('i32.mul')
                if x.lev > 0: asm.append('local.get $' + x.name)
                elif x.lev == MemAbs: asm.append('i32.const ' + str(x.adr))
                asm.append('i32.add')
                if x.tp.base == Int: 
                    asm.append('i32.load')
                    if op == SUM: asm.append('i32.add')
                    if op == MAX: 
                        asm.append('f32.convert_i32_s') 
                        asm.append('f32.max')
                elif x.tp.base == Float:
                    asm.append('f32.load')
                    if op == SUM: asm.append('f32.add')
                    elif op == MAX: asm.append('f32.max')
                elif x.tp.base == Double:
                    asm.append('f64.load') 
                    if op == SUM: asm.append('f64.add')
                    elif op == MAX: asm.append('f64.max')
                i = i + 1  
        if op == MAX and x.tp.base == Int: asm.append('i32.trunc_f32_s')
        x = Var(x.tp.base); x.lev = Stack
    return x    

In [13]:
def genIndex(x, y):
    # x[y], assuming x.tp is Array and x is global Var, local Var
    # and y is Const, local Var, global Var, stack Var
    if x.lev == MemAbs and type(y) == Const: 
        x.adr += (y.val - x.tp.lower) * x.tp.base.size
        x.tp = x.tp.base
    else:
        loadItem(y)
        if x.tp.lower != 0:
            asm.append('i32.const ' + str(x.tp.lower))
            asm.append('i32.sub')
        asm.append('i32.const ' + str(x.tp.base.size))
        asm.append('i32.mul')
        if x.lev > 0: asm.append('local.get $' + x.name)
        elif x.lev == MemAbs: asm.append('i32.const ' + str(x.adr))
        asm.append('i32.add')
        x = Var(x.tp.base)
        if x.tp in (Int, Bool, Float, Double) or type(x.tp) == Set: x.lev = MemInd
        else: x.lev = Stack
    return x

In [14]:
def genSelect(x, f):
    # x.f, assuming x.tp is Record, f is Field, and x.lev is Stack, MemInd or is > 0
    if x.lev == MemAbs: x.adr += f.offset
    elif x.lev == Stack:
        asm.append('i32.const ' + str(f.offset))
        asm.append('i32.add')
    elif x.lev > 0:
        asm.append('local.get $' + x.name) # parameter or local reference
        asm.append('i32.const ' + str(f.offset))
        asm.append('i32.add')
        x.lev = Stack
    else: mark('WASM: select?')
    x.tp = f.tp
    return x

In [15]:
def genLeftAssign(x):
    if x.lev == MemAbs: asm.append('i32.const ' + str(x.adr))
    elif x.lev > 0 and type(x.tp) in (Array, Record):
        asm.append('local.get $' + x.name)
    return x

In [16]:
def genRightAssign(x, typ = None):
    loadItem(x, typ); y = Var(x.tp); y.lev = Stack; return y

Procedure `genAssign(x, y)` generates different code for `x := y`, provided `x` is `Var`, `Ref` and `y` is `Var`, `Ref`. It generate different code based on the type of x.

In [17]:
def genAssign(x, y):
    loadItem(y, x.tp)
    if x.lev == Global: asm.append('global.set $' + x.name)
    elif x.lev > 0:
        if type(x.tp) in (Array, Record):
            asm.append('i32.const ' + str(x.tp.size))
            asm.append('memory.copy')
        else: asm.append('local.set $' + x.name)
    else:
        if type(x.tp) in (Array, Record):
            asm.append('i32.const ' + str(x.tp.size))
            asm.append('memory.copy')
        else: 
            if x.tp == Int: # if x is Int
                asm.append('i32.store')
            elif x.tp == Double: # if x is Double
                asm.append('f64.store')
            elif x.tp == Float: # if x is Float
                asm.append('f32.store')

In [18]:
def genProgEntry(ident):
    global curlev
    curlev = curlev + 1
    asm.append('(global $_memsize (mut i32) i32.const ' + str(memsize) + ')')
    asm.append('(func $program')

def genProgExit(x):
    global curlev
    curlev = curlev - 1
    asm.append('(memory ' + str(memsize // 2** 16 + 1) + ')\n(start $program)\n)')
    return '\n'.join(l for l in asm)

def genProcStart(ident, fp, rp):
    global curlev
    if curlev > 0: mark('WASM: no nested procedures')
    curlev = curlev + 1
    asm.append('(func $' + ident + ' ' + ' '.join('(param $' + e.name + ' f32)' if e.tp == Float \
                else '(param $' + e.name + ' f64)' if e.tp == Double else '(param $' + e.name + ' i32)' for e in fp) \
              + ' ' + ' '.join('(result f32)' if e.tp == Float else '(result f64)' if e.tp ==Double else \
                                                                           '(result i32)'  for e in rp) \
              + ('\n' if len(rp) > 0 else '') + '\n'.join('(local $' + e.name + ' f32)' if e.tp == Float \
                     else '(local $' + e.name + ' f64)' if e.tp == Double else '(local $' + e.name + ' i32)' for e in rp))
    return rp

def genProcEntry(ident, para, local):
    pl = (para if para else []) + local
    if any(type(l) == Var and type(l.tp) in (Array, Record) for l in pl):
        asm.append('(local $_mp i32)')
        asm.append('global.get $_memsize')
        asm.append('local.set $_mp')
    for l in pl:
        if type(l) == Var and type(l.tp) in (Array, Record):
            asm.append('global.get $_memsize')
            asm.append('local.tee $' + l.name)
            asm.append('i32.const ' + str(l.tp.size))
            asm.append('i32.add')
            asm.append('global.set $_memsize')

def genProcExit(x, para, local):
    global curlev
    curlev = curlev - 1
    pl = (para if para else []) + local
    if any(type(l) == Var and type(l.tp) in (Array, Record) for l in pl):
        asm.append('local.get $_mp')
        asm.append('global.set $_memsize')
    if para: asm.append('\n'.join('local.get $' + e.name for e in para))
    asm.append(')')

def genActualPara(ap, fp, n):
    if ap.tp in {Int, Bool, Double, Float} or type(ap.tp) == Set: loadItem(ap)
    else: # a.tp is Array, Record
        if ap.lev > 0: asm.append('local.get $' + ap.name)
        elif ap.lev == MemAbs: asm.append('i32.const ' + str(ap.adr))
        elif ap.lev != Stack: mark('WASM: actual parameter?')

def genCall(rp, pr, ap): # result (or None), procedure, actual parameters
    asm.append('call $' + pr.name)
    for r in reversed(rp): 
        if (r.tp == Double):
            y = Var(Double)
        elif (r.tp == Float): 
            y = Var(Float)
        else:
            y = Var(Int)        
        y.lev = Stack; genAssign(r, y)

# generate different read method for standard read procedure according to the different type
def genRead(x):    
    if x.tp == Int: 
        asm.append('call $readInt'); y = Var(Int)
    elif x.tp == Float: 
        asm.append('call $readFloat'); y = Var(Float)
    elif x.tp == Double: 
        asm.append('call $readDouble'); y = Var(Double)  
    y.lev = -1; genAssign(x, y)
    
# generate different write method for standard write procedure according to different type
def genWrite(x):
    if x.tp == Int:
        asm.append('call $writeInt')
    elif x.tp == Float:
        asm.append('call $writeFloat')
    elif x.tp == Double:
        asm.append('call $writeDouble')
    
def genWriteln():
    asm.append('call $writeln')

def genSeq(x, y):
    pass

def genThen(x):
    loadItem(x)
    asm.append('if')
    return x

def genIfThen(x, y):
    asm.append('end')

def genElse(x, y):
    asm.append('else')

def genIfElse(x, y, z):
    asm.append('end')

def genWhile():
    asm.append('loop')

def genDo(x):
    loadItem(x)
    asm.append('if')
    return x

def genWhileDo(t, x, y):
    asm.append('br 1')
    asm.append('end')
    asm.append('end')